# PyCity Schools Analysis

In [119]:
# Dependencies and Setup
import pandas as pd
import numpy as np
from pathlib import Path

# File to Load
school_data_to_load = Path("Resources/schools_complete.csv")
student_data_to_load = Path("Resources/students_complete.csv")

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

In [121]:
# Calculate the total number of unique schools
school_count = school_data_complete["school_name"].unique()
unique_schools = len(school_count)
unique_schools

15

In [123]:
# Calculate the total number of students
student_count = school_data_complete["student_name"].count()
student_count

39170

In [124]:
# Calculate the total budget
total_budget_df = school_data_complete[["school_name","budget"]]
total_budget_df = total_budget_df.drop_duplicates()
total_budget = total_budget_df["budget"].sum()
total_budget

24649428

In [125]:
# Calculate the average (mean) math score
average_math_score = school_data_complete["math_score"].mean()
average_math_score

78.98537145774827

In [126]:
# Calculate the average (mean) reading score
average_reading_score = school_data_complete["reading_score"].mean()
average_reading_score

81.87784018381414

In [127]:
# Calculate the percentage of students who passed math (math scores greather than or equal to 70)
passing_math_count = school_data_complete[(school_data_complete["math_score"] >= 70)].count()["student_name"]
passing_math_percentage = passing_math_count / float(student_count) * 100
passing_math_percentage

74.9808526933878

In [129]:
# Calculate the percentage of students who passed reading 
passing_reading_count = school_data_complete[(school_data_complete["reading_score"] >= 70)].count()["student_name"]
passing_reading_percentage = passing_reading_count / float(student_count) * 100
passing_reading_percentage

85.80546336482001

In [131]:
# Calculate the percentage of students that passed math and reading
passing_math_reading_count = school_data_complete[
    (school_data_complete["math_score"] >= 70) & (school_data_complete["reading_score"] >= 70)
].count()["student_name"]
overall_passing_rate = passing_math_reading_count /  float(student_count) * 100
overall_passing_rate

65.17232575950983

In [134]:
# Create a high-level snapshot of the district's key metrics in a DataFrame
district_summary_df = pd.DataFrame([{"Total Schools": unique_schools, 
                               "Total Students": student_count, 
                               "Total Budget": total_budget,
                               "Average Math Score": average_math_score,
                               "Average Reading Score": average_reading_score,
                               "% Pass Math": passing_math_percentage,
                               "% Pass Reading": passing_reading_percentage,
                               "% Overall Passing": overall_passing_rate
                               }])
# Formatting
district_summary_df.loc[:, "Total Students"] = district_summary_df["Total Students"].map('{:,.0f}'.format)
district_summary_df.loc[:, "Total Budget"] = "$" + district_summary_df["Total Budget"].map('{:,.2f}'.format)

# Display the DataFrame
district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Pass Math,% Pass Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


## School Summary

In [150]:
# Select school type for each school
school_name_and_type = school_data_complete[["school_name","type"]]
school_name_and_type = school_name_and_type.drop_duplicates()
#school_name_and_type
# Select all school types 
school_type = pd.DataFrame(school_data_complete["type"].unique())

# Display the school types
school_type


,0
0,District
1,Charter


In [141]:
# Calculate the total student count per school
per_school_counts = school_data_complete.groupby(["school_name"])

per_school_counts_df = pd.DataFrame(per_school_counts["student_name"].count())

# Rename Columns in DataFrame
per_school_counts_df = per_school_counts_df.rename(columns={"school_name": "school_name", 
                                                            "student_name": "student_count"})

# Display the total student count per school
per_school_counts_df

,student_count
school_name,
Bailey High School,4976
Cabrera High School,1858
Figueroa High School,2949
Ford High School,2739
Griffin High School,1468
Hernandez High School,4635
Holden High School,427
Huang High School,2917
Johnson High School,4761


In [157]:
# Calculate the total school budget and per capita spending per school

per_school_budget = school_data_complete[["school_name","budget"]]
per_school_budget = per_school_budget.drop_duplicates()

# Merge the total student count per school with the per school budget 
budget_and_per_capita = pd.merge(per_school_counts_df, per_school_budget, how="left", on=["school_name", "school_name"])

# Caclulate the per student budget
budget_and_per_capita["per_school_capita"] = budget_and_per_capita["budget"] / budget_and_per_capita["student_count"]

# Select columns from DataFrame
budget_and_per_capita = budget_and_per_capita.loc[:,["school_name","budget","per_school_capita"]]

# Display the total school budget and per capita spending per school
budget_and_per_capita

,school_name,budget,per_school_capita
0,Bailey High School,3124928,628.0
1,Cabrera High School,1081356,582.0
2,Figueroa High School,1884411,639.0
3,Ford High School,1763916,644.0
4,Griffin High School,917500,625.0
5,Hernandez High School,3022020,652.0
6,Holden High School,248087,581.0
7,Huang High School,1910635,655.0
8,Johnson High School,3094650,650.0
9,Pena High School,585858,609.0


In [159]:
# Calculate the average test scores per school
per_school_test_scores = school_data_complete.groupby(["school_name","type"])

per_test_scores_df = pd.DataFrame(
    per_school_test_scores[["math_score","reading_score"]].mean())

# Rename the columns in the DataFrame
per_test_scores_df = per_test_scores_df.rename(columns={"school_name": "school_name",
                                                         "math_score": "average_math_score", 
                                                         "reading_score": "average_reading_score"})

# Display the average test scores per school
per_test_scores_df

,,average_math_score,average_reading_score
school_name,type,,
Bailey High School,District,77.048432,81.033963
Cabrera High School,Charter,83.061895,83.975780
Figueroa High School,District,76.711767,81.158020
Ford High School,District,77.102592,80.746258
Griffin High School,Charter,83.351499,83.816757
Hernandez High School,District,77.289752,80.934412
Holden High School,Charter,83.803279,83.814988
Huang High School,District,76.629414,81.182722
Johnson High School,District,77.072464,80.966394


In [176]:
# Calculate the number of students per school with math scores >= 70, reading scores >= 70, both math and reading with scores >= 70

# Define criteria 
students_passing_reading = school_data_complete[(school_data_complete["reading_score"] >= 70)]
students_passing_math = school_data_complete[(school_data_complete["math_score"] >= 70)]
students_passing_both = school_data_complete[(school_data_complete["math_score"] >= 70) & (school_data_complete["reading_score"] >= 70)]

# Counts number of students per school passing reading, passing math, and passing both
school_count_passing_reading = students_passing_reading.groupby(["school_name"]).count()
school_count_passing_math = students_passing_math.groupby(["school_name"]).count()
school_count_passing_both = students_passing_both.groupby(["school_name"]).count()

# Select required columns only
school_student_count_passing_reading = school_count_passing_reading.loc[:,["reading_score"]]
school_student_count_passing_math = school_count_passing_math.loc[:,["math_score"]]
school_student_count_passing_both = school_count_passing_both.loc[:,["math_score"]]
school_student_count_passing_both = school_student_count_passing_both.rename(columns={"school_name": "school_name",
                                                         "math_score": "passing_both_count"})

# Merge the total students per school DataFrames with % passing DataFrames
school_merge_reading = pd.merge(per_school_counts_df, school_student_count_passing_reading, how="left", on=["school_name"])
school_merge_math = pd.merge(per_school_counts_df, school_student_count_passing_math, how="left", on=["school_name"])
school_merge_both = pd.merge(per_school_counts_df, school_student_count_passing_both, how="left", on=["school_name"])

# Caculate the % passing reading, % passing math, and % overall passing
school_merge_reading["% Passing Reading"] = school_merge_reading["reading_score"] / school_merge_reading["student_count"] * 100
school_merge_math["% Passing Math"] = school_merge_math["math_score"] / school_merge_math["student_count"] * 100
school_merge_both["% Overall Passing"] = school_merge_both["passing_both_count"] / school_merge_both["student_count"] * 100

# Merge all the DataFrames together
school_merge = pd.merge(pd.merge(school_merge_reading,school_merge_math, how="left", on=["school_name"]),school_merge_both, how="left", on=["school_name"])

# Select required columns only
school_merge_loc = school_merge.loc[:,["% Passing Math","% Passing Reading","% Overall Passing"]]

# Display the % passing reading, % passing math, and % overall passing
school_merge_loc

,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,
Bailey High School,66.680064,81.933280,54.642283
Cabrera High School,94.133477,97.039828,91.334769
Figueroa High School,65.988471,80.739234,53.204476
Ford High School,68.309602,79.299014,54.289887
Griffin High School,93.392371,97.138965,90.599455
Hernandez High School,66.752967,80.862999,53.527508
Holden High School,92.505855,96.252927,89.227166
Huang High School,65.683922,81.316421,53.513884
Johnson High School,66.057551,81.222432,53.539172


In [184]:
# Create a DataFrame called `per_school_summary` with columns for the calculations above.
per_school_summary = school_name_and_type.merge(per_school_counts_df,on='school_name').merge(budget_and_per_capita,on='school_name').merge(per_test_scores_df,on='school_name').merge(school_merge_loc,on='school_name')

# Formatting
per_school_summary["budget"] = per_school_summary["budget"].map("${:,.2f}".format)
per_school_summary["per_school_capita"] = per_school_summary["per_school_capita"].map("${:,.2f}".format)
per_school_summary = per_school_summary.sort_values("school_name")

# Rename Columns in DataFrame
per_school_summary = per_school_summary.rename(columns={"school_name": "School Name", 
                                                        "type": "School Type",
                                                        "student_count": "Total Student",
                                                        "budget": "Total School Budget",
                                                        "per_school_capita": "Per Student Budget",
                                                        "average_math_score": "Average Math Score",
                                                        "average_reading_score": "Average Reading Score",
                                                        "% Passing Math":  "% Passing Math",
                                                        "% Passing Reading": "% Passing Reading",
                                                        "% Overall Passing": "% Overall Passing"                                                                                 
                                                        })

# Display the DataFrame
per_school_summary

,School Name,School Type,Total Student,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
7,Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
6,Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
1,Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
13,Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
4,Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
3,Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
8,Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
0,Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
12,Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172
9,Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Highest-Performing Schools (by % Overall Passing)

In [186]:
# Sort the schools by `% Overall Passing` in descending order and display the top 5 rows.
top_schools = per_school_summary.sort_values("% Overall Passing",ascending=False)
top_schools.head(5)

,School Name,School Type,Total Student,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
6,Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
14,Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
4,Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
5,Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
9,Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541
